# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fdf7c996df0>
├── 'a' --> tensor([[-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095]])
└── 'x' --> <FastTreeValue 0x7fdf7c996f70>
    └── 'c' --> tensor([[-1.1050,  0.2551,  0.2071,  0.6705],
                        [-0.1900, -1.2521,  0.0025, -1.1509],
                        [-0.8992, -0.0948, -0.5590, -0.1117]])

In [4]:
t.a

tensor([[-0.3994,  0.0602, -0.6460],
        [ 0.5714, -0.2030,  0.5095]])

In [5]:
%timeit t.a

80.3 ns ± 0.232 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fdf7c996df0>
├── 'a' --> tensor([[-0.4609, -1.7587, -0.6691],
│                   [ 0.0799,  0.5912,  0.0304]])
└── 'x' --> <FastTreeValue 0x7fdf7c996f70>
    └── 'c' --> tensor([[-1.1050,  0.2551,  0.2071,  0.6705],
                        [-0.1900, -1.2521,  0.0025, -1.1509],
                        [-0.8992, -0.0948, -0.5590, -0.1117]])

In [7]:
%timeit t.a = new_value

84.4 ns ± 0.105 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095]]),
    x: Batch(
           c: tensor([[-1.1050,  0.2551,  0.2071,  0.6705],
                      [-0.1900, -1.2521,  0.0025, -1.1509],
                      [-0.8992, -0.0948, -0.5590, -0.1117]]),
       ),
)

In [10]:
b.a

tensor([[-0.3994,  0.0602, -0.6460],
        [ 0.5714, -0.2030,  0.5095]])

In [11]:
%timeit b.a

66 ns ± 0.133 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5740, -0.4757,  1.0811],
               [-0.8665,  0.0288,  0.2384]]),
    x: Batch(
           c: tensor([[-1.1050,  0.2551,  0.2071,  0.6705],
                      [-0.1900, -1.2521,  0.0025, -1.1509],
                      [-0.8992, -0.0948, -0.5590, -0.1117]]),
       ),
)

In [13]:
%timeit b.a = new_value

546 ns ± 3.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

885 ns ± 0.974 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

149 µs ± 801 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

149 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fdf7c9a19d0>
├── 'a' --> tensor([[[-0.3994,  0.0602, -0.6460],
│                    [ 0.5714, -0.2030,  0.5095]],
│           
│                   [[-0.3994,  0.0602, -0.6460],
│                    [ 0.5714, -0.2030,  0.5095]],
│           
│                   [[-0.3994,  0.0602, -0.6460],
│                    [ 0.5714, -0.2030,  0.5095]],
│           
│                   [[-0.3994,  0.0602, -0.6460],
│                    [ 0.5714, -0.2030,  0.5095]],
│           
│                   [[-0.3994,  0.0602, -0.6460],
│                    [ 0.5714, -0.2030,  0.5095]],
│           
│                   [[-0.3994,  0.0602, -0.6460],
│                    [ 0.5714, -0.2030,  0.5095]],
│           
│                   [[-0.3994,  0.0602, -0.6460],
│                    [ 0.5714, -0.2030,  0.5095]],
│           
│                   [[-0.3994,  0.0602, -0.6460],
│                    [ 0.5714, -0.2030,  0.5095]]])
└── 'x' --> <FastTreeValue 0x7fdf7c9a1ac0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

34.1 µs ± 58.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fde95077100>
├── 'a' --> tensor([[-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095],
│                   [-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095],
│                   [-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095],
│                   [-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095],
│                   [-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095],
│                   [-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095],
│                   [-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095],
│                   [-0.3994,  0.0602, -0.6460],
│                   [ 0.5714, -0.2030,  0.5095]])
└── 'x' --> <FastTreeValue 0x7fde95091b80>
    └── 'c' --> tensor([[-1.1050,  0.2551,  0.2071,  0.6705],
                        [-0.1900, -1.2521,  0.0025, -1.1509],
                 

In [23]:
%timeit t_cat(trees)

31.7 µs ± 35.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.3 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3994,  0.0602, -0.6460],
                [ 0.5714, -0.2030,  0.5095]],
       
               [[-0.3994,  0.0602, -0.6460],
                [ 0.5714, -0.2030,  0.5095]],
       
               [[-0.3994,  0.0602, -0.6460],
                [ 0.5714, -0.2030,  0.5095]],
       
               [[-0.3994,  0.0602, -0.6460],
                [ 0.5714, -0.2030,  0.5095]],
       
               [[-0.3994,  0.0602, -0.6460],
                [ 0.5714, -0.2030,  0.5095]],
       
               [[-0.3994,  0.0602, -0.6460],
                [ 0.5714, -0.2030,  0.5095]],
       
               [[-0.3994,  0.0602, -0.6460],
                [ 0.5714, -0.2030,  0.5095]],
       
               [[-0.3994,  0.0602, -0.6460],
                [ 0.5714, -0.2030,  0.5095]]]),
    x: Batch(
           c: tensor([[[-1.1050,  0.2551,  0.2071,  0.6705],
                       [-0.1900, -1.2521,  0.0025, -1.1509],
                       [-0.8992, -0.0948, -0.5590, -0.1117]],
         

In [26]:
%timeit Batch.stack(batches)

85.2 µs ± 194 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095],
               [-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095],
               [-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095],
               [-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095],
               [-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095],
               [-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095],
               [-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095],
               [-0.3994,  0.0602, -0.6460],
               [ 0.5714, -0.2030,  0.5095]]),
    x: Batch(
           c: tensor([[-1.1050,  0.2551,  0.2071,  0.6705],
                      [-0.1900, -1.2521,  0.0025, -1.1509],
                      [-0.8992, -0.0948, -0.5590, -0.1117],
                      [-1.1050,  0.2551,  0.2071,  0.6705],
                      [-0.1900, -

In [28]:
%timeit Batch.cat(batches)

155 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

382 µs ± 5.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
